# Battle Royale Environment Trainer
This notebook is for training Battle Royale agents. MADDPG is used for training the agents.

## Setup Environment Dependencies

In [1]:
import sys
from gym_unity.envs import UnityEnv

print("Python version:")
print(sys.version)
print(sys.executable)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")

Python version:
3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
/Users/adhipradhana/anaconda3/envs/unity-battle-royale/bin/python


## Start Environment

In [2]:
# Environment name
# Remember to put battle royale environment configuration within the config folder
env_name = "environment/battle-royale-static"

env = UnityEnv(env_name, worker_id=5, use_visual=False, multiagent=True)

print(str(env))

INFO:mlagents_envs:Connected new brain:
PlayerBrain?team=0
INFO:gym_unity:2 agents within environment.


<UnityEnv instance>


/Users/adhipradhana/anaconda3/envs/unity-battle-royale/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


## Testing Model

### Setup Algorithm Dependencies

In [3]:
import torch
import numpy as np
import random

from utils.MADDPG import MADDPG

### Model Variables

In [4]:
random_seed = random.randint(0,1000000)
n_states = env.observation_space.shape[0]
n_actions = env.action_space.shape[0]
n_agents = env.number_agents
n_episode = 100
max_steps = 100
buffer_capacity = 1000000
batch_size = 1000
episodes_before_train = 100

In [5]:
# setup seed
torch.manual_seed(random_seed)
np.random.seed(random_seed)

maddpg = MADDPG(n_agents, n_states, n_actions, batch_size, buffer_capacity, episodes_before_train)

FloatTensor = torch.cuda.FloatTensor if maddpg.use_cuda else torch.FloatTensor

## Loading Model

In [6]:
import os

path = os.path.join(os.getcwd(), 'checkpoint', 'Time_2020-04-03_08.32.21.183897_NAgent_2_Episode_500.pth')
maddpg.load(path, map_location='cpu')

## Run Model

In [ ]:
print("Testing model...")
for i_episode in range(n_episode):
    # reset environment
    obs = env.reset()
    obs = np.stack(obs)
    
    # convert observation to tensor
    if isinstance(obs, np.ndarray):
        obs = torch.from_numpy(obs).float()
    
    total_reward = 0.0
    for i_step in range(max_steps):
        obs = obs.type(FloatTensor)
        actions = maddpg.select_action(obs).data.cpu()
        actions_list = actions.tolist()
        
        obs_, reward, done, _ = env.step(actions_list)
        
        reward = torch.FloatTensor(reward).type(FloatTensor)
        obs_ = np.stack(obs_)
        obs_ = torch.from_numpy(obs_).float()
        if i_step != max_steps - 1:
            next_obs = obs_
        else:
            next_obs = None

        total_reward += reward.sum()     
        obs = next_obs

        # check if done
        if True in done:
            print(done)
            break

    maddpg.episode_done += 1
    print("Episode: {}, reward = {}".format(i_episode, total_reward))

Testing model...
[True, True]
Episode: 0, reward = 0.3500000238418579
[True, True]
Episode: 1, reward = 1.0499999523162842
[True, True]
Episode: 2, reward = 0.6399999856948853
[True, True]
Episode: 3, reward = 0.36000001430511475
[True, True]
Episode: 4, reward = 1.2200000286102295
[True, True]
Episode: 5, reward = 0.6800001263618469
[True, True]
Episode: 6, reward = 1.190000057220459
[True, True]
Episode: 7, reward = 0.4800000488758087
[True, True]
Episode: 8, reward = 1.4800002574920654
[True, True]
Episode: 9, reward = 1.4600001573562622


## Close Environment

In [22]:
env.close()

INFO:mlagents_envs:Environment shut down with return code 0.
